In [20]:
# Import libraries
import pandas as pd 
import requests
import os 
import json
import pprint
import numpy as np

In [21]:
# Read original df with neighborhood and coordinates
%store -r nyc_df

In [22]:
nyc_df.columns = ['Neighborhood', 'Borough', 'Longitude', 'Latitude']
nyc_df

,Neighborhood,Borough,Longitude,Latitude
0,Wakefield,Bronx,-73.847201,40.894705
1,Co-op City,Bronx,-73.829939,40.874294
2,Eastchester,Bronx,-73.827806,40.887556
3,Fieldston,Bronx,-73.905643,40.895437
4,Riverdale,Bronx,-73.912585,40.890834
...,...,...,...,...
301,Hudson Yards,Manhattan,-74.000111,40.756658
302,Hammels,Queens,-73.805530,40.587338
303,Bayswater,Queens,-73.765968,40.611322
304,Queensbridge,Queens,-73.945631,40.756091


In [23]:
# Set the key
fsq_api_key = os.environ['FOURSQUARE_API_KEY']

In [24]:
lat_list = nyc_df['Latitude'].tolist()
lon_list = nyc_df['Longitude'].tolist()
name_list = nyc_df['Neighborhood'].tolist

In [25]:
# venue info lists
venues_name = []
venues_cat = []
venues_lat = []
venues_lon = []
venues_ngb = []
venues_zip = []

# Base url 
base_url = 'https://api.foursquare.com/v3/places/search'
fields = 'name%2Cgeocodes%2Clocation%2Ccategories' # Replaced comma with '%2C'

# Iterate through df
for i in range(len(lat_list)):
    # define URL parameters
    ll = f'{lat_list[i]}%2C{lon_list[i]}' 
    params = f'?ll={ll}&radius=500&categories=12000&fields={fields}&limit=50'
    url = base_url+params
    #print(url)
    
    # set headers
    headers = {
        "accept": "application/json",
        "Authorization": fsq_api_key}
    
    # Make the GET request
    response = requests.get(url, headers=headers)
       
    for venue in response.json()['results']:
        venues_name.append(venue['name'])
        venues_cat.append(venue['categories'][0]['name'])
        venues_lat.append(venue['geocodes']['main']['latitude'])
        venues_lon.append(venue['geocodes']['main']['longitude'])
        try:
            venues_ngb.append(venue['location']['neighborhood'][0])
        except: 
            venues_ngb.append(np.nan)
        try:
            venues_zip.append(venue['location']['postcode'])
        except:
            venues_zip.append(np.nan)

In [26]:
print(len(venues_name))
print(len(venues_cat))
print(len(venues_lat))
print(len(venues_lon))
print(len(venues_ngb))
print(len(venues_zip))

9836
9836
9836
9836
9836
9836


In [27]:
nyc_gov = pd.DataFrame(list(zip(
    venues_name, venues_cat, venues_lat, venues_lon, venues_ngb, venues_zip)))

In [28]:
nyc_gov.columns = [
    'Venue', 'Category', 'Latitude', 'Longitude', 'Neighborhood', 'ZIP'
]

In [29]:
nyc_gov.shape

(9836, 6)

In [30]:
nyc_gov.head(5)

,Venue,Category,Latitude,Longitude,Neighborhood,ZIP
0,Public School 87,Elementary School,40.895310,-73.845988,Wakefield,10466
1,Little Stars Preschool,Preschool,40.891571,-73.844276,Edenwald,10466
2,Redeemer Lutheran Church,Church,40.897004,-73.851570,Wakefield,10466
3,Philadelphia Seventh Day Adventist Church,Church,40.891661,-73.851244,Williamsbridge,10466
4,Mount Saint Michael Academy,Private School,40.897394,-73.842691,Wakefield,10466


In [31]:
nyc_gov.isnull().sum()

Venue             0
Category          0
Latitude          0
Longitude         0
Neighborhood    643
ZIP               1
dtype: int64

In [32]:
%store nyc_gov

Stored 'nyc_gov' (DataFrame)


In [33]:
# Export df to a csv file
nyc_gov.to_csv('nyc_gov.csv', index=False)

# Test Area

In [82]:
base_url = 'https://api.foursquare.com/v3/places/search'
fields = 'name%2Cgeocodes%2Clocation%2Ccategories' # Replaced comma with '%2C'
ll = f'{lat_list[1]}%2C{lon_list[1]}' 
params = f'?ll={ll}&radius=500&categories=15000&fields={fields}&limit=50'
url = base_url+params
print(url)

headers = {
"accept": "application/json",
"Authorization": fsq_api_key}
        
# Make the GET request
response = requests.get(url, headers=headers)

https://api.foursquare.com/v3/places/search?ll=40.87429419303012%2C-73.82993910812398&radius=500&categories=15000&fields=name%2Cgeocodes%2Clocation%2Ccategories&limit=50


In [83]:
response.status_code

200

In [127]:
response.json()['results']

[{'categories': [{'id': 15019,
    'name': 'Mental Health Clinic',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_doctorsoffice_',
     'suffix': '.png'}}],
  'geocodes': {'main': {'latitude': 40.620457, 'longitude': -74.080547},
   'roof': {'latitude': 40.620457, 'longitude': -74.080547}},
  'location': {'address': '101 Warren St',
   'census_block': '360850029003000',
   'country': 'US',
   'dma': 'New York',
   'formatted_address': '101 Warren St, New York, NY 10304',
   'locality': 'New York',
   'neighborhood': ['Stapleton'],
   'postcode': '10304',
   'region': 'NY'},
  'name': 'Staten Island Mental Health'},
 {'categories': [{'id': 15016,
    'name': 'Medical Center',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
     'suffix': '.png'}}],
  'geocodes': {'main': {'latitude': 40.614241, 'longitude': -74.084439}},
  'location': {'address': '531 Targee St',
   'census_block': '360850040033000',
   'country': 'US',

In [55]:
venues_name = []
venues_cat = []
venues_lat = []
venues_lon = []

In [56]:
for venue in response.json()['results']:
    venues_name.append(venue['name'])
    venues_cat.append(venue['categories'][0]['name'])
    venues_lat.append(venue['geocodes']['main']['latitude'])
    venues_lon.append(venue['geocodes']['main']['longitude'])

In [57]:
print(venues_name)
print(venues_cat)
print(venues_lat)
print(venues_lon)

['Riverdale Lactation Center', 'Coagulation Sciences']
['Hospital', 'Health and Medicine']
[40.892949, 40.895337]
[-73.903551, -73.911439]


In [75]:
response.json()['results'][0]['name']['neighborhood']

['Stapleton']